In [164]:
import pandas as pd
import re
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [165]:
salons_df = pd.read_csv("HairSalons.csv")

In [166]:
salons_df.head()

,Stars,Page_URL,Page_Title,Field2,Services
0,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/urban-betty-austin-3?...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,$$,Blowout Services ...
1,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/method-hair-austin?os...,Method.Hair - 83 Photos & 420 Reviews - Hair S...,$$,NaN
2,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/red-stella-hair-salon...,Red Stella Hair Salon - 339 Photos & 281 Revie...,$$,Acne Treatments ...
3,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/topaz-salon-austin?os...,Topaz Salon - 29 Photos & 243 Reviews - Hair S...,$$,NaN
4,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/garbo-a-salon-and-spa...,Garbo A Salon and Spa - 54 Photos & 239 Review...,$$,Beards & Mustaches ...


In [167]:
def get_salon_name(row):
    page_title = row["Page_Title"].split(" - ") # split page title by the dash
    return page_title[0] # The salon name is the first of the split
def get_number_of_reviews(row):
    page_title = row["Page_Title"].split(" - ")
    photos_and_reviews = page_title[1]
    split_photos_and_reviews = photos_and_reviews.split(" ")
    #print(split_photos_and_reviews)
    if(len(split_photos_and_reviews) < 4): # if there isn't enough data here, then there likely aren't any reviews
        return 0
    while True:
        try:
            reviews = int(split_photos_and_reviews[3])
            return (reviews)
        except ValueError:
            return 0
def get_address(row):
    page_title = row["Page_Title"].split(" - ")
    for i in page_title:
        if ("Austin, TX" in i): # look for strings that contain the city name as it'll most likely be the address
            return i
def get_price(row):
    price = row["Field2"]
    return price

In [168]:
# Get the number of stars for a company
def get_stars(row):
    stars_row = row["Stars"]
    if (pd.isnull(stars_row)):
        return -1
    result = re.search('alt="(.*)"', stars_row) # splits the data into ---4.5 star rating" height="303" width="84---
    split_result = result.group(1).split(" ") # Split up the previous text from before
    stars = split_result[0]
    return(stars)

In [169]:
def get_services(row):
    services_ugly = row["Services"] # get the services stuff in its ugliest form
    if (type(services_ugly) != str):
        return ""
    services_split = services_ugly.split("  ")
    services = ""
    for i in services_split:
        keep = 0
        while True:
            try:
                if (i[0] == " "):
                    i = i[1:]
                    keep = 1
                elif (i[0] != " "):
                    i = i
                    keep = 1
                if (services == "" and keep):
                    services = i
                elif (keep):
                    services = services + ", " + i
                break
            except:
                break
    return services

In [170]:
salons_df["Title"] = salons_df.apply(get_salon_name, axis=1)
salons_df["Number_of_reviews"] = salons_df.apply(get_number_of_reviews, axis=1)
salons_df["Address"] = salons_df.apply(get_address, axis=1)
salons_df["Star Rating"] = salons_df.apply(get_stars, axis=1)
salons_df["Price"] = salons_df.apply(get_price, axis=1)
salons_df["Service list"] = salons_df.apply(get_services, axis=1)
salons_df.head(5)

,Stars,Page_URL,Page_Title,Field2,Services,Title,Number_of_reviews,Address,Star Rating,Price,Service list
0,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/urban-betty-austin-3?...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,$$,Blowout Services ...,Urban Betty,537,"1206 W 38th St, Rosedale, Austin, TX",4.5,$$,"Blowout Services, Bridal Services, Coloring Se..."
1,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/method-hair-austin?os...,Method.Hair - 83 Photos & 420 Reviews - Hair S...,$$,NaN,Method.Hair,420,"1601 E 5th St, East Austin, Austin, TX",4.5,$$,
2,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/red-stella-hair-salon...,Red Stella Hair Salon - 339 Photos & 281 Revie...,$$,Acne Treatments ...,Red Stella Hair Salon,281,"5117 N Lamar Blvd, Austin, TX",4.5,$$,"Acne Treatments, Anti-aging Treatments, Blowou..."
3,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/topaz-salon-austin?os...,Topaz Salon - 29 Photos & 243 Reviews - Hair S...,$$,NaN,Topaz Salon,243,"1509 S Lamar Blvd, Bouldin Creek, Austin, TX",4.5,$$,
4,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/garbo-a-salon-and-spa...,Garbo A Salon and Spa - 54 Photos & 239 Review...,$$,Beards & Mustaches ...,Garbo A Salon and Spa,239,"7739 Northcross Dr, Allandale, Austin, TX",4.5,$$,"Beards & Mustaches, Blowout Services, Bridal S..."


In [159]:
salons_df.iloc[:,10]

0      Blowout Services, Bridal Services, Coloring Se...
1                                                       
2      Acne Treatments, Anti-aging Treatments, Blowou...
3                                                       
4      Beards & Mustaches, Blowout Services, Bridal S...
5      Blowout Services, Bridal Services, Coloring Se...
6                                                       
7      Coloring Services, Hair Treatment Services, Ha...
8                                                       
9                                                       
10                                                      
11     Acne Treatments, Anti-aging Treatments, Beards...
12     Beards & Mustaches, Blowout Services, Bridal S...
13                                                      
14     Blowout Services, Bridal Services, Coloring Se...
15                                                      
16     Anti-aging Treatments, Blowout Services, Brida...
17     Blowout Services, Bridal

In [173]:
salons_df.head()

,Stars,Page_URL,Page_Title,Field2,Services,Title,Number_of_reviews,Address,Star Rating,Price,Service list
0,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/urban-betty-austin-3?...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,$$,Blowout Services ...,Urban Betty,537,"1206 W 38th St, Rosedale, Austin, TX",4.5,$$,"Blowout Services, Bridal Services, Coloring Se..."
1,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/method-hair-austin?os...,Method.Hair - 83 Photos & 420 Reviews - Hair S...,$$,NaN,Method.Hair,420,"1601 E 5th St, East Austin, Austin, TX",4.5,$$,
2,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/red-stella-hair-salon...,Red Stella Hair Salon - 339 Photos & 281 Revie...,$$,Acne Treatments ...,Red Stella Hair Salon,281,"5117 N Lamar Blvd, Austin, TX",4.5,$$,"Acne Treatments, Anti-aging Treatments, Blowou..."
3,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/topaz-salon-austin?os...,Topaz Salon - 29 Photos & 243 Reviews - Hair S...,$$,NaN,Topaz Salon,243,"1509 S Lamar Blvd, Bouldin Creek, Austin, TX",4.5,$$,
4,"<img class=""offscreen"" src=""https://s3...",https://www.yelp.com/biz/garbo-a-salon-and-spa...,Garbo A Salon and Spa - 54 Photos & 239 Review...,$$,Beards & Mustaches ...,Garbo A Salon and Spa,239,"7739 Northcross Dr, Allandale, Austin, TX",4.5,$$,"Beards & Mustaches, Blowout Services, Bridal S..."


Reshape into a newer, cleaner dataframe to load to the database

In [172]:
salons_clean = salons_df.copy()
salons_clean.drop(["Page_Title", "Stars", "Page_URL", "Services", "Field2"], axis=1, inplace = True)
salons_clean.head()

,Title,Number_of_reviews,Address,Star Rating,Price,Service list
0,Urban Betty,537,"1206 W 38th St, Rosedale, Austin, TX",4.5,$$,"Blowout Services, Bridal Services, Coloring Se..."
1,Method.Hair,420,"1601 E 5th St, East Austin, Austin, TX",4.5,$$,
2,Red Stella Hair Salon,281,"5117 N Lamar Blvd, Austin, TX",4.5,$$,"Acne Treatments, Anti-aging Treatments, Blowou..."
3,Topaz Salon,243,"1509 S Lamar Blvd, Bouldin Creek, Austin, TX",4.5,$$,
4,Garbo A Salon and Spa,239,"7739 Northcross Dr, Allandale, Austin, TX",4.5,$$,"Beards & Mustaches, Blowout Services, Bridal S..."


Once all the data is put into a pandas dataframe, convert that to a SQL database

In [190]:
f=open("db.txt", "r")
contents = 0
if not(contents):
    contents = f.read()
username = contents.split("\n")[0]
password = contents.split("\n")[1]
dbname = 'salons_db'

In [193]:
engine = create_engine('postgres://%s:%s@localhost/%s'%(username,password, dbname))

In [194]:
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [195]:
salons_clean.to_sql('salons_data_table', engine, if_exists='replace')

In [199]:
con = None
con = psycopg2.connect(database = dbname, user = username, password = password, port=5432, host= "/var/run/postgresql/")

# query:
sql_query = """
SELECT * FROM salons_data_table;
"""
salons_data_from_sql = pd.read_sql_query(sql_query,con)
salons_data_from_sql.head()

,index,Title,Number_of_reviews,Address,Star Rating,Price,Service list
0,0,Urban Betty,537,"1206 W 38th St, Rosedale, Austin, TX",4.5,$$,"Blowout Services, Bridal Services, Coloring Se..."
1,1,Method.Hair,420,"1601 E 5th St, East Austin, Austin, TX",4.5,$$,
2,2,Red Stella Hair Salon,281,"5117 N Lamar Blvd, Austin, TX",4.5,$$,"Acne Treatments, Anti-aging Treatments, Blowou..."
3,3,Topaz Salon,243,"1509 S Lamar Blvd, Bouldin Creek, Austin, TX",4.5,$$,
4,4,Garbo A Salon and Spa,239,"7739 Northcross Dr, Allandale, Austin, TX",4.5,$$,"Beards & Mustaches, Blowout Services, Bridal S..."
